In [ ]:
import math
import timeit
import random
import sympy
import warnings
from random import randint, seed
import sys
from ecpy.curves import Curve,Point
from ecpy.keys import ECPublicKey, ECPrivateKey
from Crypto.Hash import SHA3_256
import requests
from Crypto.Cipher import AES
from Crypto import Random
from Crypto.Util.Padding import pad
from Crypto.Util.Padding import unpad
import re
import json
API_URL = 'http://cryptlygos.pythonanywhere.com'

def sign(k, m):
	global P
	R = k * P
	r = R.x % n

	h = int.from_bytes(SHA3_256.new(str(m).encode("utf-8") + r.to_bytes((r.bit_length() + 7) // 8, byteorder="big")).digest(), byteorder="big") % n
	s = (sL * h + k) % n

	return h, s

def ver(s, h, m, qA): #verify
	global P
	sP = int(s) * P
	hA = int(h) * qA
	V = sP - hA
	v = V.x % n
	hnew = int.from_bytes(SHA3_256.new(m.encode("utf-8") + v.to_bytes((v.bit_length() + 7) // 8, byteorder= "big")).digest(), byteorder= "big") % n
	return h == hnew

stuID = 23631

# the curve
E = Curve.get_curve('secp256k1')
n = E.order
P = E.generator

#HERE CREATE A LONG TERM KEY
sL = 23927729476774547655045008832834504310921797729497501158914766546192068080855
ql = sL * P

lkey = ql

#

k = random.randint(1, n - 1)
h, s = sign(k, str(stuID))

#server's long term key
QSer_long = Point(0xc1bc6c9063b6985fe4b93be9b8f9d9149c353ae83c34a434ac91c85f61ddd1e9 , 0x931bd623cf52ee6009ed3f50f6b4f92c564431306d284be7e97af8e443e69a8c, E)

# HERE GENERATE A EPHEMERAL KEY 
sA = random.randint(0, n - 1)
qA = sA * P

ekey = qA

try:
	#REGISTRATION
	mes = {'ID':stuID, 'h': h, 's': s, 'LKEY.X': lkey.x, 'LKEY.Y': lkey.y}
	response = requests.put('{}/{}'.format(API_URL, "RegStep1"), json = mes)		
	if((response.ok) == False): raise Exception(response.json())
	print(response.json())

	print("Enter verification code which is sent to you: ")	
	code = int(input())

	mes = {'ID':stuID, 'CODE': code}
	response = requests.put('{}/{}'.format(API_URL, "RegStep3"), json = mes)
	if((response.ok) == False): raise Exception(response.json())
	print(response.json())
	
	#STS PROTOCOL

	mes = {'ID': stuID, 'EKEY.X': ekey.x, 'EKEY.Y': ekey.y}
	response = requests.put('{}/{}'.format(API_URL, "STSStep1&2"), json = mes)
	if((response.ok) == False): raise Exception(response.json())
	res=response.json()

	#calculate T,K,U
	QB = Point(res["SKEY.X"], res["SKEY.Y"], E)
	T = sA * QB
	U = str(T.x) + str(T.y) + "BeYourselfNoMatterWhatTheySay"
	K = int.from_bytes(SHA3_256.new(U.encode("utf-8")).digest(), byteorder= "big")

	#W1 Part4

	W1 = str(qA.x) + str(qA.y) + str(QB.x) + str(QB.y)

	h, s = sign(K, W1)

	pt = ("s" + str(s) + "h" + str(h)).encode("utf-8")
	aes = AES.new(K.to_bytes((K.bit_length() + 7) // 8, byteorder= "big"), AES.MODE_CTR)
	Y1 = aes.nonce + aes.encrypt(pt)
	Y1Int = int.from_bytes(Y1, byteorder= "big")

	###Send encrypted-signed keys and retrive server's signed keys
	mes = {'ID': stuID, 'FINAL MESSAGE': Y1Int}
	print(mes)
	response = requests.put('{}/{}'.format(API_URL, "STSStep4&5"), json = mes)
	if((response.ok) == False): raise Exception(response.json()) 
	ctext= response.json() 

	#W2 Part5
	'''
	W2 = str(QB.x) + str(QB.y) + str(qA.x) + str(qA.y)

	h, s = sign(sL, W2)

	pt = ("s" + str(s) + "h" + str(h)).encode("utf-8")
	Y2 = AES.new(K.to_bytes((K.bit_length() + 7) // 8, byteorder= "big"), AES.MODE_CTR)
	Y2 = Y2.nonce + Y2.encrypt(pt)
	Y2Int = int.from_bytes(Y2, byteorder= "big")
	'''
	#Decrypt 
	QS = Point(0xc1bc6c9063b6985fe4b93be9b8f9d9149c353ae83c34a434ac91c85f61ddd1e9, 0x931bd623cf52ee6009ed3f50f6b4f92c564431306d284be7e97af8e443e69a8c, E)
	Y2 = ctext.to_bytes((ctext.bit_length() + 7) // 8, byteorder= "big")
	#Y2 = Y2[len(aes.nonce):]

	aes = AES.new(K.to_bytes((K.bit_length() + 7) // 8, byteorder= "big"), AES.MODE_CTR, nonce= Y2[0:8])
	Y2 = aes.decrypt(Y2[8:])
	Y2 = Y2.decode("utf-8")

	W2 = str(QB.x) + str(QB.y) + str(qA.x) + str(qA.y)

	#verify
	s = int(Y2[1:Y2.find("h")])
	h = int(Y2[Y2.find("h") + 1:])
	print(ver(s, h, W2, QS))

	#get a message from server for 
	mes = {'ID': stuID}
	response = requests.get('{}/{}'.format(API_URL, "STSStep6"), json=mes)
	ctext= response.json()         
	
	#Decrypt
	ctext = ctext.to_bytes((ctext.bit_length() + 7) // 8, byteorder= "big")

	W3 = ctext[8:]
	aes = AES.new(K.to_bytes((K.bit_length() + 7) // 8, byteorder= "big"), AES.MODE_CTR, nonce= ctext[0:8])
	ptext = aes.decrypt(W3).decode("utf-8")
	
	randNum = int(ptext[ptext.rfind(" "):]) + 1
	mes = ptext[:ptext.rfind(" ") + 1] + str(randNum)

	#Add 1 to random to create the new message and encrypt it
	aes = AES.new(K.to_bytes((K.bit_length() + 7) // 8, byteorder= "big"), AES.MODE_CTR)
	m = aes.encrypt(mes.encode("utf-8"))

	ct = int.from_bytes(aes.nonce + m, byteorder= "big")
	
	#send the message and get response of the server
	mes = {'ID': stuID, 'ctext': ct}
	response = requests.put('{}/{}'.format(API_URL, "STSStep7&8"), json = mes)
	ctext= response.json()         

	ctext = ctext.to_bytes((ctext.bit_length() + 7) // 8, byteorder= "big")

	W3 = ctext[8:]
	aes = AES.new(K.to_bytes((K.bit_length() + 7) // 8, byteorder= "big"), AES.MODE_CTR, nonce= ctext[0:8])
	ptext = aes.decrypt(W3).decode("utf-8")
	
	print(ptext)

except Exception as e:
	print(e)
